In [1]:
import geojson

import gaia

# Initialize Girder connection
apikey = 'LcxASyej0yUl9Pr7mgAmf3kj2a7a92SsemTln8ps'
datastore = gaia.connect(apikey=apikey)

In [4]:
# Create gaia dataset for girder-hosted file
girder_path = 'Public/DEM_bare_earth.tif'  # (~4GB)

# API option 1: create internal url
datastore_url = datastore.lookup_url(girder_path)
# print('datastore url:', datastore_url)
dataset = gaia.create(datastore_url)

# API option 2: pass in tuple as the data source
# data_source = (datastore, 'Public/DEM_bare_earth.tif')
# dataset = gaia.create(data_source)

# Todo is there a better api for these options?
dataset.set_opacity(0.5)
dataset.set_mapnik_style({
    'band': 1,
    'max': 5000,
    'min': 2000,
    'palette': 'matplotlib.Plasma_6',
    'scheme': 'linear',
})


# meta = dataset.get_metadata()
# import json
# # print(json.dumps(meta, sort_keys=True, indent=2))
# print('Input dataset width: {}, height: {}'.format(meta['width'], meta['height']))
#gaia.show(dataset, representation='outline')
scene = gaia.show(dataset)
scene.zoom = 10
display(scene)

tiles_url http://localhost:8989/api/v1/item/5bca474261700e00077b2220/tiles/zxy/{z}/{x}/{y}?encoding=PNG&projection=EPSG:3857&style=%7B%22band%22%3A+1%2C+%22min%22%3A+2000%2C+%22scheme%22%3A+%22linear%22%2C+%22palette%22%3A+%22matplotlib.Plasma_6%22%2C+%22max%22%3A+5000%7D


scene(center={'x': -106.9669195224971, 'y': 39.03667206832321}, layout=Layout(align_self='stretch', height='40…

In [5]:
import ipywidgets as widgets
z = widgets.FloatSlider(min=1, max=16, value=10, layout=dict(width='95%'))
newlink = widgets.jslink((z, 'value'), (scene, 'zoom'))
display(z)

FloatSlider(value=10.0, layout=Layout(width='95%'), max=16.0, min=1.0)

In [5]:
# Generate small crop geometry
bounds = meta.get('bounds',{}).get('coordinates')[0]
assert bounds, 'Dataset bounds missing'
# print()
# print(bounds)

# Compute center coordinates
x = (bounds[0][0] + bounds[2][0]) / 2.0
y = (bounds[0][1] + bounds[2][1]) / 2.0

# Use smll percentage of height & width
dx = 0.005 * (bounds[2][0] - bounds[0][0])
dy = 0.005 * (bounds[2][1] - bounds[0][1])
rect = [
    [x,y], [x, y-dy], [x-dx, y-dy], [x-dx, y], [x,y]
]
# print()
# print(rect)

# Must pass rectangle in as a LIST, in order to get geom formatted the way resgeodata uses
crop_geom = geojson.Polygon([rect])
print()
print(crop_geom)


{"coordinates": [[[-106.9669195224971, 38.94564248418757], [-106.9669195224971, 38.944732188346215], [-106.96771495471891, 38.944732188346215], [-106.96771495471891, 38.94564248418757], [-106.9669195224971, 38.94564248418757]]], "type": "Polygon"}


In [6]:
# Crop the dataset
from gaia import preprocess
cropped_dataset = preprocess.crop2(dataset, crop_geom, name='crop100m.tif')

Deleting existing (item crop100m.tif)
 1 Checking job 5bccefd161700e013c22a54c status... 2
 2 Checking job 5bccefd161700e013c22a54c status... 2
 3 Checking job 5bccefd161700e013c22a54c status... 2
 4 Checking job 5bccefd161700e013c22a54c status... 2
 5 Checking job 5bccefd161700e013c22a54c status... 2
 6 Checking job 5bccefd161700e013c22a54c status... 2
 7 Checking job 5bccefd161700e013c22a54c status... 2
 8 Checking job 5bccefd161700e013c22a54c status... 2
 9 Checking job 5bccefd161700e013c22a54c status... 2
10 Checking job 5bccefd161700e013c22a54c status... 2
11 Checking job 5bccefd161700e013c22a54c status... 2
12 Checking job 5bccefd161700e013c22a54c status... 2
13 Checking job 5bccefd161700e013c22a54c status... 3


In [7]:
print()
cropped_meta = cropped_dataset.get_metadata()
print('Cropped dataset width {}, height {}'.format(
    cropped_meta['width'], cropped_meta['height']))
#print(cropped_meta)


Cropped dataset width 143, height 207


In [8]:
# Todo Update gaia.show() for girder-resident datasets
scene = gaia.show(cropped_dataset, representation="outline")
scene.zoom = 11
scene

In createFeature


scene(center={'y': 38.945660385510074, 'x': -106.96731536343347}, layout=Layout(align_self='stretch', height='…